In [ ]:
!pip install -q streamlit openai python-dotenv pydantic

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
%%writefile legal_case_analysis.py
import os
import streamlit as st
from openai import OpenAI
from pydantic import BaseModel, Field
import json
import re

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Streamlit page configuration
st.set_page_config(
    page_title="Legal Case Analyzer",
    page_icon="⚖️",
    layout="wide",
    initial_sidebar_state="expanded",
)

# Custom CSS for styling
st.markdown("""
<style>
    div.stButton > button[kind="primary"] {
        background-color: #1E88E5;
        color: white;
    }
    div.stSlider > div[data-baseweb="slider"] > div > div {
        background-color: #42A5F5 !important;
    }
    .stSelectbox div[data-baseweb="select"] > div {
        border-color: #0D47A1;
    }
    h1, h2, h3 {
        color: #0D47A1;
    }
    .sidebar .sidebar-content {
        background-color: #f0f2f5;
    }
</style>
""", unsafe_allow_html=True)

st.title("AI-Powered Legal Case Analyzer")
st.markdown("### Analyze legal cases with multi-agent AI")

# Sidebar for case parameters and jurisdiction
st.sidebar.header("Case Parameters")

# Jurisdiction selection
jurisdiction = st.sidebar.selectbox(
    "Jurisdiction",
    options=["Federal", "State", "International"],
    help="Select the legal jurisdiction for the case."
)

# Additional case metadata
case_type = st.sidebar.selectbox(
    "Case Type",
    options=["Criminal", "Civil", "Contract", "Tort", "Other"],
    help="Select the type of legal case."
)

case_complexity = st.sidebar.slider(
    "Case Complexity",
    min_value=1,
    max_value=10,
    value=5,
    help="Rate the complexity of the case (1 = Simple, 10 = Highly Complex)."
)

case_facts = st.sidebar.text_area(
    "Enter Case Facts",
    placeholder="Describe the facts of the legal case...",
    height=200
)

with st.sidebar.expander("LLM Settings"):
    triage_model_name = st.text_input("Triage Model", value="gpt-3.5-turbo")
    defense_model_name = st.text_input("Defense Model", value="gpt-4o-mini")
    prosecution_model_name = st.text_input("Prosecution Model", value="gpt-4-turbo")
    judge_model_name = st.text_input("Judge Model", value="gpt-4o")

# Pydantic model for structured verdict
class CaseVerdict(BaseModel):
    verdict: str = Field(description="The final verdict (e.g., 'Guilty', 'Not Guilty', 'Dismissed')")
    confidence: float = Field(description="Confidence level in the verdict (0 to 1)")
    reasoning: str = Field(description="Explanation for the verdict")
    key_precedents: str = Field(description="Relevant legal precedents or principles considered")

# Utility function to extract JSON from text
def extract_json_from_text(text):
    try:
        json_match = re.search(r'```json\s*([\s\S]*?)\s*```', text)
        if json_match:
            json_str = json_match.group(1)
            return json.loads(json_str)
        return json.loads(text)
    except:
        return None

# OpenAI Agent class
class OpenAIAgent:
    def __init__(self, model_name, system_prompt, api_key):
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.client = OpenAI(api_key=api_key)

    def run(self, user_prompt, message_history=None):
        messages = [{"role": "system", "content": self.system_prompt}]

        if message_history:
            messages.extend(message_history)

        messages.append({"role": "user", "content": user_prompt})

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            temperature=0.7
        )

        return response.choices[0].message.content

def run_case_analysis():
    if not OPENAI_API_KEY:
        st.error("Please set the OpenAI API key in the code")
        return
    if not case_facts:
        st.error("Please enter case facts")
        return

    # Initialize OpenAI client
    try:
        with st.spinner("Initializing AI models..."):
            # Triage Agent (gpt-3.5-turbo)
            triage_system_prompt = f"""
            You are a Triage Agent for a legal case analysis system. Your role is to analyze the provided case facts: {case_facts},
            jurisdiction: {jurisdiction}, case type: {case_type}, and case complexity: {case_complexity}.
            Categorize the case and provide a summary to direct it to the appropriate agents (Prosecution, Defense, Judge).
            Filter out irrelevant information and highlight key issues relevant to the jurisdiction and case type.
            Provide a structured response with:
            - Case Category (string): The primary legal category
            - Key Issues (string): Main legal issues to be addressed
            - Agent Assignment (string): Which agents should handle the case
            - Summary (string): Brief summary of the case for other agents
            Format your response as plain text.
            """
            triage_agent = OpenAIAgent(triage_model_name, triage_system_prompt, OPENAI_API_KEY)

            # Defense Agent (gpt-4o-mini)
            defense_system_prompt = f"""
            You are an experienced defense attorney operating under {jurisdiction} law. Your role is to build a strong defense argument based on the provided case facts: {case_facts}.
            Focus on mitigating factors, reasonable doubt, and legal defenses that could favor the defendant.
            Consider the case type ({case_type}) and jurisdiction-specific legal principles.
            Provide a persuasive and factual argument to support the defendant's case.
            """
            defense_agent = OpenAIAgent(defense_model_name, defense_system_prompt, OPENAI_API_KEY)

            # Prosecution Agent (gpt-4-turbo)
            prosecution_system_prompt = f"""
            You are an experienced prosecutor operating under {jurisdiction} law. Your role is to analyze the provided case facts: {case_facts}.
            Identify weaknesses in the defense's position, highlight incriminating evidence, and argue for conviction.
            Consider the case type ({case_type}) and jurisdiction-specific legal principles.
            Provide a critical and factual argument to challenge the defense.
            """
            prosecution_agent = OpenAIAgent(prosecution_model_name, prosecution_system_prompt, OPENAI_API_KEY)

            # Judge Agent (gpt-4o)
            judge_system_prompt = f"""
            You are an impartial judge operating under {jurisdiction} law. Your role is to evaluate the provided case facts: {case_facts},
            along with the defense and prosecution arguments, considering the case type ({case_type}) and complexity ({case_complexity}).
            Make a final determination on the case outcome, considering jurisdiction-specific legal principles, evidence, and arguments presented.
            You MUST structure your response as a valid JSON object with the following fields:
            - verdict (string): The final verdict (e.g., 'Guilty', 'Not Guilty', 'Dismissed')
            - confidence (number): Confidence level in the verdict (0 to 1)
            - reasoning (string): Explanation for the verdict
            - key_precedents (string): Relevant legal precedents or principles considered
            Format your JSON response within ```json ``` code blocks.
            """
            judge_agent = OpenAIAgent(judge_model_name, judge_system_prompt, OPENAI_API_KEY)

            st.success("Models initialized successfully")
    except Exception as e:
        st.error(f"Error initializing AI models: {e}")
        return

    tab1, tab2, tab3, tab4 = st.tabs(["Triage Report", "Defense Argument", "Prosecution Argument", "Final Verdict"])

    # Triage Agent
    with tab1:
        with st.spinner("Generating triage report..."):
            st.subheader("Triage Report")
            triage_prompt = "Analyze and categorize the case based on the provided facts, jurisdiction, case type, and complexity."
            triage_result = triage_agent.run(triage_prompt)
            st.markdown(triage_result)

    # Defense Agent
    with tab2:
        with st.spinner("Generating defense argument..."):
            st.subheader("Defense Argument")
            defense_prompt = "Provide a detailed defense argument based on the case facts and jurisdiction."
            defense_result = defense_agent.run(defense_prompt)
            st.markdown(defense_result)
            defense_message = {"role": "assistant", "content": defense_result}

    # Prosecution Agent
    with tab3:
        with st.spinner("Generating prosecution argument..."):
            st.subheader("Prosecution Argument")
            prosecution_prompt = "Provide a detailed prosecution argument identifying weaknesses in the defense."
            prosecution_result = prosecution_agent.run(prosecution_prompt)
            st.markdown(prosecution_result)
            prosecution_message = {"role": "assistant", "content": prosecution_result}

    # Judge Agent
    with tab4:
        with st.spinner("Determining final verdict..."):
            st.subheader("Final Verdict")
            final_question = f"""Based on the case facts, jurisdiction ({jurisdiction}), case type ({case_type}), complexity ({case_complexity}), and arguments presented, determine the case outcome.

Please provide your verdict in JSON format with these fields:
- verdict (string)
- confidence (number between 0 and 1)
- reasoning (string)
- key_precedents (string)"""
            combined_messages = [defense_message, prosecution_message]
            judge_result = judge_agent.run(final_question, combined_messages)

            raw_response = judge_result
            st.text("Model Response:")
            with st.expander("Show raw response"):
                st.markdown(raw_response)

            json_data = extract_json_from_text(raw_response)

            if json_data:
                try:
                    verdict = CaseVerdict(**json_data)
                    verdict_color = "green" if verdict.verdict == "Not Guilty" else "red" if verdict.verdict == "Guilty" else "orange"

                    st.markdown(f"""
                    ### Verdict: <span style='color:{verdict_color}'>{verdict.verdict}</span>

                    **Confidence**: {verdict.confidence:.2f}

                    **Reasoning**:
                    {verdict.reasoning}

                    **Key Precedents or Principles**:
                    {verdict.key_precedents}
                    """, unsafe_allow_html=True)
                except Exception as e:
                    st.error(f"Error validating response format: {e}")
                    st.markdown("### Verdict Summary")
                    st.markdown(raw_response)
            else:
                st.error("Could not parse a structured verdict. Showing raw response:")
                st.markdown(raw_response)

            st.markdown("---")
            st.caption("**Disclaimer**: This tool provides automated legal analysis for educational purposes only. Always consult with a professional attorney for legal advice.")

if st.button("Analyze Case", type="primary", use_container_width=True):
    run_case_analysis()
else:
    st.info("👈 Enter case facts, jurisdiction, case type, and complexity in the sidebar and click 'Analyze Case' to get started.")

    st.markdown(f"""
    ### How this works
    1. Enter the case facts, select jurisdiction, case type, and complexity in the sidebar
    2. Click 'Analyze Case' to start the analysis
    3. The Triage Agent (using {triage_model_name}) categorizes the case and prepares a summary
    4. The Defense Agent (using {defense_model_name}) generates a defense argument
    5. The Prosecution Agent (using {prosecution_model_name}) generates a prosecution argument
    6. The Judge Agent (using {judge_model_name}) evaluates all arguments and delivers a final verdict
    7. A structured verdict will be provided based on the case facts, jurisdiction, and arguments

    This tool uses multiple AI models to simulate a balanced legal analysis process, providing structured outcomes tailored to the case and jurisdiction.
    """)


In [ ]:
!streamlit run legal_case_analysis.py &>/content/logs.txt &

In [ ]:
!curl ifconfig.me

In [ ]:
!npx localtunnel --port 8501